In [28]:
import numpy as np
import pandas as pd
import os
import glob

### Combine files in one DataFrame

In [63]:
# get path of all .csv files (history from API crawling)
all_files = glob.glob(os.path.join(os.path.dirname(os.path.abspath('')),"raw_data","api_test1", "*.csv"))

# Define column names 
col_names=['icao24','callsign','origin_country','time_position','last_contact','long','lat','baro_altitude','on_ground','velocity',       
'true_track','vertical_rate','sensors','geo_altitude','squawk','spi','position_source', "drop", "time"]

# Concatenate to large Dataframe
df = pd.concat((pd.read_csv(f, index_col=0, names=col_names, header=0) for f in all_files)) 

### Filter for passenger flights and on ground

In [ ]:
# to be done

In [56]:
df.groupby("icao24")[["on_ground"]].sum().sort_values(by=["on_ground"], ascending=False).head(20)

,on_ground
icao24,
5113fa,44.0
5113fc,44.0
474801,44.0
344043,44.0
3411c7,44.0
344115,44.0
474806,44.0
76eb00,44.0
4b1808,42.0


In [59]:
df[df.icao24=="c1e847"]

,icao24,callsign,origin_country,time_position,last_contact,long,lat,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source,drop,time
5041,c1e847,BMN74,Canada,1652824444.0,1.652824e+09,-79.6059,43.6769,NaN,1.0,9.77,45.00,NaN,NaN,NaN,NaN,0.0,0,17.0,1652824461
5176,c1e847,BMN74,Canada,1652823532.0,1.652824e+09,-79.6085,43.6752,NaN,1.0,4.37,225.00,NaN,NaN,NaN,NaN,0.0,0,17.0,1652823534
4181,c1e847,BMN74,Canada,1652832353.0,1.652832e+09,-79.6485,43.6811,NaN,1.0,2.31,61.88,NaN,NaN,NaN,NaN,0.0,0,NaN,1652832362
5519,c1e847,BMN74,Canada,1652820757.0,1.652821e+09,-79.6488,43.6810,NaN,1.0,8.75,47.81,NaN,NaN,NaN,NaN,0.0,0,17.0,1652820761
5000,c1e847,BMN74,Canada,1652824997.0,1.652825e+09,-79.6180,43.6589,NaN,1.0,13.89,230.62,NaN,NaN,NaN,NaN,0.0,0,17.0,1652825222
4567,c1e847,NaN,Canada,1652829943.0,1.652830e+09,-79.6286,43.6863,NaN,1.0,7.72,315.00,NaN,NaN,NaN,NaN,0.0,0,17.0,1652829949
4869,c1e847,NaN,Canada,1652827534.0,1.652828e+09,-79.6002,43.6713,NaN,1.0,7.72,14.06,NaN,NaN,NaN,NaN,0.0,0,17.0,1652827805
4829,c1e847,BMN74,Canada,1652828445.0,1.652828e+09,-79.5954,43.6730,NaN,1.0,3.09,95.62,NaN,NaN,NaN,NaN,0.0,0,17.0,1652828748
4736,c1e847,BMN74,Canada,1652829037.0,1.652829e+09,-79.5983,43.6815,NaN,1.0,6.69,64.69,NaN,NaN,NaN,NaN,0.0,0,17.0,1652829047
4630,c1e847,BMN74,Canada,1652829588.0,1.652830e+09,-79.6160,43.6760,NaN,1.0,3.34,315.00,NaN,NaN,NaN,NaN,0.0,0,17.0,1652829655
